**INSTALL DEPENDENCIES**

In [ ]:
!pip install -r /content/drive/MyDrive/podcast-project/requirements.txt

# !pip install -q openai-whisper
# !pip install -q librosa pydub noisereduce pyloudnorm soundfile
# !pip install -q spacy tqdm jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=06c548353315a361bccd6d15c78a09bc8e6a6dc5c988d8b8389629958e77ec8b
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**DEFINE PATHS**

In [ ]:
AUDIO_RAW = "/content/drive/MyDrive/podcast-project/data/audio_raw"
TRANS_PROC = "/content/drive/MyDrive/podcast-project/data/transcripts_processed"
TMP_AUDIO = "/content/drive/MyDrive/podcast-project/data/audio_tmp"

import os
# Create the directory if it does not already exist
os.makedirs(TMP_AUDIO, exist_ok=True)
os.makedirs(TRANS_PROC, exist_ok=True)


**LOAD WHISPER**

In [ ]:
import whisper  # OpenAI Whisper library for automatic speech recognition (ASR)
import torch    # PyTorch library used for tensor operations and hardware acceleration

# Use GPU (CUDA) if available for faster inference
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Load the Whisper "tiny" mode
model = whisper.load_model("tiny").to(DEVICE)


Using device: cpu


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 120MiB/s]


**AUDIO PREPROCESSING + CHUNKING**

In [ ]:
import librosa                 # Audio analysis library (loading, resampling, feature extraction)
import soundfile as sf         # Read/write audio files (WAV)
import shutil                  # File operations (copying, moving, deleting files)

import noisereduce as nr       # Noise reduction
import pyloudnorm as pyln      # Loudness measurement and normalization

from pydub import AudioSegment                 # High-level audio manipulation (format conversion, slicing)
from pydub.silence import detect_nonsilent     # Detect non-silent regions in audio (speech/activity detection)


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
def preprocess_and_chunk(mp3_path, episode_id, chunk_sec=30):
    # Create a temporary working directory for this episode
    episode_dir = os.path.join(TMP_AUDIO, f"episode_{episode_id}")
    os.makedirs(episode_dir, exist_ok=True)

    # MP3 → mono → 16kHz(16000) WAV
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_channels(1).set_frame_rate(16000)

    # Save the cleaned audio as a WAV file
    conditioned = os.path.join(episode_dir, "conditioned.wav")
    audio.export(conditioned, format="wav")

    # Load the WAV file into memory for processing
    y, sr = librosa.load(conditioned, sr=16000)

    # Background Noise reduction (like hiss or hum)
    y = nr.reduce_noise(y=y, sr=sr, stationary=True)

    # Measure and Normalize Loudness
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(y)
    y = pyln.normalize.loudness(y, loudness, -14.0)   # -14 LUFS → best loudness for clear spoken audio(Podcast)

    # Trim long silence at the start or end
    nonsilent = detect_nonsilent(
        AudioSegment.from_wav(conditioned),
        min_silence_len=600,    # silence longer than 0.6 seconds
        silence_thresh=-40      #–40 dB → very quiet (room noise, background hiss)
    )

    if nonsilent:
        start, end = nonsilent[0][0], nonsilent[-1][1]
        y = y[int(start/1000*sr):int(end/1000*sr)]
        offset = start / 1000     # Real audio starts this many seconds later.
    else:
        offset = 0.0      # If no silence was found, assume audio starts at the beginning

    # Chunking
    chunks = []
    samples = chunk_sec * sr    # number of samples in one chunk
    current = offset            # start time of the first chunk

    for i in range(0, len(y), samples):
        chunk = y[i:i+samples]
        # Stop if the remaining audio is too short
        if len(chunk) < samples * 0.4:
            break

        path = os.path.join(episode_dir, f"chunk_{i//samples:03d}.wav")
        sf.write(path, chunk, sr)

        chunks.append({
            "path": path,
            "start_sec": round(current, 2)
        })
        current += chunk_sec

    return chunks, episode_dir


**TEXT CLEANING + SENTENCE SEGMENTATION**

In [ ]:
import re, spacy
nlp = spacy.load("en_core_web_sm")

# Create a pattern to find common filler words in speech
# re.I means case-insensitive (Uh, um, LIKE, etc.)
FILLER = re.compile(r"\b(uh|um|you know|like)\b", re.I)   # \b means “word boundary” so we don’t remove parts of real words

def clean_and_segment(text):
    text = FILLER.sub("", text)   # Remove filler words from the text
    return [s.text.strip() for s in nlp(text).sents if s.text.strip()]    # Split text into sentences, remove extra spaces, and drop empty ones



**WHISPER TRANSCRIPTION**

In [ ]:
import json
from tqdm import tqdm   # Show a progress bar for loops

In [ ]:
def transcribe_episode(ep, chunks):
    segments = []   # List to store each sentence with start and end time
    full_text = []  # List to store all text for the full episode transcript

    # Loop through each audio chunk
    for c in tqdm(chunks, desc=f"Episode {ep}"):

        # Transcribe the audio chunk using Whisper
        result = model.transcribe(
            c["path"],                  # path to chunk audio file
            language="en",              # English language
            fp16=(DEVICE == "cuda"),    # use GPU
            verbose=False               # do not print extra logs
        )

        # Loop through Whisper's time-based segments
        for seg in result["segments"]:

            # Convert chunk-relative times to original episode times
            abs_start = round(seg["start"] + c["start_sec"], 2)
            abs_end = round(seg["end"] + c["start_sec"], 2)

            sentences = clean_and_segment(seg["text"])    # Clean text and split it into sentences

            # Store each sentence with its timing
            for s in sentences:
                segments.append({
                    "start": abs_start,
                    "end": abs_end,
                    "text": s
                })
                full_text.append(s)

    # Path where the final transcript JSON will be saved
    out_path = os.path.join(
        TRANS_PROC, f"episode_{ep}_whisper.json"
    )

    # Save transcript data to a JSON file
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(
            {
                "episode_id": ep,
                "segments": segments,
                "text": " ".join(full_text)
            },
            f, indent=2, ensure_ascii=False
        )


**DRIVER LOOP**

In [ ]:
import os

# Get all episode numbers from MP3 filenames

episodes = []

for ep in range(21, 31):
    mp3_path = os.path.join(AUDIO_RAW, f"{ep}.mp3")
    if os.path.exists(mp3_path):
        episodes.append(ep)
    else:
        print(f"Missing file: {ep}.mp3")

print("Episodes to process:", episodes)


Episodes to process: [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [ ]:
for ep in episodes:
    print(f"\nEpisode {ep}")    # episode which is being processed

    mp3 = os.path.join(AUDIO_RAW, f"{ep}.mp3")
    if not os.path.exists(mp3):
        print("Missing audio")
        continue    # skip this episode and move to the next one

    chunks, temp_dir = preprocess_and_chunk(mp3, ep)    # Preprocess the audio and split it into chunks
    transcribe_episode(ep, chunks)                      # Transcribe all chunks and save the transcript

    # Clean temp audio
    shutil.rmtree(temp_dir, ignore_errors=True)

    print(f"Episode {ep} completed & cleaned")



Episode 21


/usr/local/lib/python3.12/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
Episode 21: 100%|██████████| 118/118 [11:54<00:00,  6.05s/it]


Episode 21 completed & cleaned

Episode 22


/usr/local/lib/python3.12/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
Episode 22: 100%|██████████| 118/118 [13:25<00:00,  6.83s/it]


Episode 22 completed & cleaned

Episode 23


...

/usr/local/lib/python3.12/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
Episode 28: 100%|██████████| 116/116 [14:29<00:00,  7.50s/it]


Episode 28 completed & cleaned

Episode 29


/usr/local/lib/python3.12/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
Episode 29: 100%|██████████| 118/118 [19:56<00:00, 10.14s/it]


Episode 29 completed & cleaned

Episode 30


/usr/local/lib/python3.12/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
  0%|          | 0/3000 [00:00<?, ?frames/s]